# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [38]:
# Imports:
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
from geoviews import opts

# Import API key
from api_keys import geoapify_key

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,57.79,82,75,11.18,RU,1721713816
1,1,bethel,41.3712,-73.4140,71.69,88,100,3.44,US,1721713816
2,2,ushuaia,-54.8000,-68.3000,35.26,69,0,5.75,AR,1721713575
3,3,san patricio,28.0170,-97.5169,84.83,75,46,6.69,US,1721713817
4,4,puerto ayora,-0.7393,-90.3518,68.56,96,95,4.00,EC,1721713817


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [40]:
%%capture --no-display

# Create a map that displays a point for every city in the city_data_df DataFrame
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='Humidity', 
    alpha=0.6, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600,
    hover_cols=['City', 'Country', 'Humidity']
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [41]:
# Check the range of values for temperature, wind speed, and cloudiness
print(city_data_df[['Max Temp', 'Wind Speed', 'Cloudiness']].describe())

         Max Temp  Wind Speed  Cloudiness
count  565.000000  565.000000  565.000000
mean    71.643858    8.094655   53.504425
std     13.619594    5.895003   39.067981
min     26.220000    0.000000    0.000000
25%     62.830000    3.780000   13.000000
50%     72.950000    6.550000   59.000000
75%     81.480000   11.180000   95.000000
max    105.030000   37.760000  100.000000


In [42]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] < 45) & 
    (city_data_df['Max Temp'] > 10) & 
    (city_data_df['Wind Speed'] < 20) & 
    (city_data_df['Cloudiness'] < 50)
]
# Display sample data
ideal_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,ushuaia,-54.8000,-68.3000,35.26,69,0,5.75,AR,1721713575
93,93,bredasdorp,-34.5322,20.0403,44.71,83,29,4.34,ZA,1721713831
178,178,sisimiut,66.9395,-53.6735,43.65,81,27,14.12,GL,1721713843
320,320,el calafate,-50.3408,-72.2768,36.77,87,27,3.51,AR,1721713863
404,404,senillosa,-39.0000,-68.4167,42.58,28,5,1.21,AR,1721713875
419,419,balcarce,-37.8462,-58.2552,44.82,79,3,1.99,AR,1721713877
536,536,patacamaya,-17.2333,-67.9167,34.92,90,20,6.98,BO,1721713895


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Copy relevant data to a new DataFrame
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column for hotel names
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,ushuaia,AR,-54.8000,-68.3000,69,
93,bredasdorp,ZA,-34.5322,20.0403,83,
178,sisimiut,GL,66.9395,-53.6735,81,
320,el calafate,AR,-50.3408,-72.2768,87,
404,senillosa,AR,-39.0000,-68.4167,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Define the search radius and base URL for Geoapify API
radius = 10000  # 10,000 meters
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 1  # We only need the first result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat, lon = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{lon},{lat},{radius}'
    params['bias'] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
bredasdorp - nearest hotel: Victoria Hotel
sisimiut - nearest hotel: Hotel SØMA
el calafate - nearest hotel: Hostería Las Avutardas
senillosa - nearest hotel: No hotel found
balcarce - nearest hotel: 646 Hotel Balcarce
patacamaya - nearest hotel: Hostal Viña del Mar


,City,Country,Lat,Lng,Humidity,Hotel Name
2,ushuaia,AR,-54.8000,-68.3000,69,Apart Hotel Aires del Beagle
93,bredasdorp,ZA,-34.5322,20.0403,83,Victoria Hotel
178,sisimiut,GL,66.9395,-53.6735,81,Hotel SØMA
320,el calafate,AR,-50.3408,-72.2768,87,Hostería Las Avutardas
404,senillosa,AR,-39.0000,-68.4167,28,No hotel found
419,balcarce,AR,-37.8462,-58.2552,79,646 Hotel Balcarce
536,patacamaya,BO,-17.2333,-67.9167,90,Hostal Viña del Mar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
points_with_hotels = hotel_df.hvplot.points(
    'Lng', 'Lat',
    size='Humidity',
    color='Humidity',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    hover_cols=['City', 'Country', 'Hotel Name'],
    title="Hotel Locations"
)
# Display the map
points_with_hotels.opts(
    opts.Points(tools=['hover'], colorbar=True)
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)